In [7]:
import pandas as pd
import numpy as np

In [8]:
person = pd.read_excel("../../Data/MasterfileCore/ProgramPersonnel.xlsx")

In [9]:
person.columns

Index(['pgm_id', 'pers_type', 'aamc_id', 'pers_name_sfx', 'pers_name_last',
       'pers_name_first', 'pers_name_mid', 'pers_deg1', 'pers_deg2',
       'pers_deg3', 'AAMC_deg1', 'AAMC_deg2', 'AAMC_deg3', 'pers_chg_dt',
       'pers_ph_num', 'pers_ph_ext', 'pers_fx_num', 'pers_e-mail',
       'pers_ama_e-mail', 'last_upd_dt', 'survey_cycle', 'last_upd_type',
       'last_upd_id', 'pers_name_correct', 'prev_director_last',
       'prev_director_first', 'prev_director_md', 'prev_director_sfx',
       'acgme_verified', 'approval_date', 'why_change', 'subscribe',
       'diff_name', 'email_notify', 'me.'],
      dtype='object')

In [3]:
ppd = pd.read_csv("../../Data/PPD/ppd_data_20200718.csv")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
directors = person[(person.pers_type=='D')]
directors = directors.fillna('None')
directors['FIRST_NAME']=[x.upper() for x in directors.pers_name_first]
directors['LAST_NAME']=[x.upper() for x in directors.pers_name_last]

In [5]:
ppd['pers_deg1']=['MD' if x==1 else 'DO' for x in ppd.MD_DO_CODE]

In [9]:
all_directors = directors[directors.aamc_id!='None'].sort_values(['survey_cycle']).drop_duplicates(['aamc_id'], keep='last')

In [10]:
all_match = pd.merge(ppd, all_directors, on=['FIRST_NAME','LAST_NAME'])
pure_match = pd.merge(ppd, all_directors, on=['FIRST_NAME','LAST_NAME']).drop_duplicates('aamc_id', keep=False)

In [11]:
duplicates = all_match[all_match.aamc_id.isin(pure_match.aamc_id)==False]

In [12]:
unmatched = all_directors[all_directors.aamc_id.isin(all_match.aamc_id)==False]

In [13]:
dupes = all_directors[all_directors.aamc_id.isin(duplicates.aamc_id)]

In [14]:
duplicates = duplicates.fillna('None')

In [15]:
matched_dict_list = []
unmatched_still = pd.DataFrame()
for row in dupes.itertuples():
    new_df = duplicates[duplicates.aamc_id == row.aamc_id]
    if row.pers_deg1 != 'None' and row.pers_deg1 != 'MPH':
        new_df = new_df[new_df.pers_deg1_x==row.pers_deg1]
    if len(new_df)>1 and row.pers_name_mid != 'None':
        if len(row.pers_name_mid)==1:
            new_df['middle']=[x[0] for x in new_df.MIDDLE_NAME]
            new_df = new_df[new_df.middle == row.pers_name_mid]
        else:
            new_df = new_df[new_df.MIDDLE_NAME==row.pers_name_mid.upper()]
    if len(new_df)==1:
        matched_dict_list.append({'aamc_id':row.aamc_id, 'me': list(new_df.ME)[0]})
    else:
        unmatched_still = pd.concat([new_df, unmatched_still])
    print("")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [16]:
NEW_MATCH = pd.DataFrame(matched_dict_list).rename(columns={'me':'ME'})

In [28]:
pd.concat([pure_match[['ME', 'aamc_id']], NEW_MATCH]).to_csv("../../Data/MasterfileCore/ME_TO_AAMC.csv", index=False)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [17]:
len(pd.concat([pure_match[['ME', 'aamc_id']], NEW_MATCH]))

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


9453

In [18]:
len(all_directors)

12321

In [19]:
9453/12321

0.7672266861456051

In [21]:
list(ppd.columns)

['ME',
 'RECORD_ID',
 'UPDATE_TYPE',
 'ADDRESS_TYPE',
 'MAILING_NAME',
 'LAST_NAME',
 'FIRST_NAME',
 'MIDDLE_NAME',
 'SUFFIX',
 'MAILING_LINE_1',
 'MAILING_LINE_2',
 'CITY',
 'STATE',
 'ZIP',
 'SECTOR',
 'CARRIER_ROUTE',
 'ADDRESS_UNDELIVERABLE_FLAG',
 'FIPS_COUNTY',
 'FIPS_STATE',
 'PRINTER_CONTROL_CODE',
 'PC_ZIP',
 'PC_SECTOR',
 'DELIVERY_POINT_CODE',
 'CHECK_DIGIT',
 'PRINTER_CONTROL_CODE_2',
 'REGION',
 'DIVISION',
 'GROUP',
 'TRACT',
 'SUFFIX_CENSUS',
 'BLOCK_GROUP',
 'MSA_POPULATION_SIZE',
 'MICRO_METRO_IND',
 'CBSA',
 'CBSA_DIV_IND',
 'MD_DO_CODE',
 'BIRTH_YEAR',
 'BIRTH_CITY',
 'BIRTH_STATE',
 'BIRTH_COUNTRY',
 'GENDER',
 'TELEPHONE_NUMBER',
 'PRESUMED_DEAD_FLAG',
 'FAX_NUMBER',
 'TOP_CD',
 'PE_CD',
 'PRIM_SPEC_CD',
 'SEC_SPEC_CD',
 'MPA_CD',
 'PRA_RECIPIENT',
 'PRA_EXP_DT',
 'GME_CONF_FLG',
 'FROM_DT',
 'TO_DT',
 'YEAR_IN_PROGRAM',
 'POST_GRADUATE_YEAR',
 'GME_SPEC_1',
 'GME_SPEC_2',
 'TRAINING_TYPE',
 'GME_INST_STATE',
 'GME_INST_ID',
 'MEDSCHOOL_STATE',
 'MEDSCHOOL_ID',
 'M

In [25]:
ppd[[
 'MPA_CD',
 'PRA_RECIPIENT',
 'PRA_EXP_DT',
 'GME_CONF_FLG',
 'FROM_DT',
 'TO_DT',
 'YEAR_IN_PROGRAM',
 'POST_GRADUATE_YEAR',
 'GME_SPEC_1',
 'GME_SPEC_2',
 'TRAINING_TYPE',
 'GME_INST_STATE',
 'GME_INST_ID',
 'MEDSCHOOL_STATE',
 'MEDSCHOOL_ID',
 'MEDSCHOOL_GRAD_YEAR',
 'NO_CONTACT_IND']].head()

,MPA_CD,PRA_RECIPIENT,PRA_EXP_DT,GME_CONF_FLG,FROM_DT,TO_DT,YEAR_IN_PROGRAM,POST_GRADUATE_YEAR,GME_SPEC_1,GME_SPEC_2,TRAINING_TYPE,GME_INST_STATE,GME_INST_ID,MEDSCHOOL_STATE,MEDSCHOOL_ID,MEDSCHOOL_GRAD_YEAR,NO_CONTACT_IND
0,OFF,NaN,NaN,Y,NaN,6302003.0,NaN,NaN,NaN,NaN,NaN,11.0,440.0,1,2,2000,NaN
1,NCL,NaN,NaN,NaN,NaN,10312006.0,NaN,NaN,NaN,NaN,NaN,48.0,9502.0,1,2,2001,NaN
2,OFF,NaN,NaN,Y,NaN,6302003.0,NaN,NaN,NaN,NaN,NaN,36.0,291.0,1,2,2000,NaN
3,OFF,NaN,NaN,Y,NaN,10312004.0,NaN,NaN,NaN,NaN,NaN,3.0,136.0,1,2,2000,NaN
4,HPP,NaN,NaN,Y,NaN,6302004.0,NaN,NaN,NaN,NaN,NaN,1.0,498.0,1,2,2000,NaN


In [31]:
ppd[(ppd.FIRST_NAME=='GERMAINE')&(ppd.LAST_NAME=='CUMMINGS')]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
173244,1221990044,A,1,1.0,GERMAINE D CUMMINGS MD,CUMMINGS,GERMAINE,DENISE,NaN,NaN,...,GA,30052,8257.0,R014,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
all_twit= pd.read_csv('../../Data/Medscape/2020-09-17/Twitter_Doctors_2020-09-17.csv')
some_twit= pd.read_excel('../../Data/Medscape/2020-09-17/Twitter_Physicians_2020-09-17.xlsx')

In [36]:
all_twit[all_twit.NAME.isin(some_twit['NAME'])==False]

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,AGE,DATE,STATE,LINK
4,Dr. Neil Blatt,NEIL,BLATT,NaN,NaN,NaN,DR.,74,09-08-2020,NY,https://www.google.com/amp/s/www.newsday.com/a...
7,Dr. Jerry Bosse,JERRY,BOSSE,NaN,NaN,NaN,DR.,79,08-31-2020,MO,https://www.stlouiscovidmemorial.com/dr-jerry-...
9,Dr. Daniel Thomas,DANIEL,THOMAS,NaN,NaN,NaN,DR.,73,08-26-2020,CA,https://m.facebook.com/notifications.php
10,Dr. Keith Daniels,KEITH,DANIELS,NaN,NaN,NaN,DR.,64,08-23-2020,MI,https://www.irakaufman.com/mobile/detail.php?i...
13,Dr. John McLaulin,JOHN,MCLAULIN,NaN,NaN,NaN,DR.,68,08-19-2020,GA,https://www.google.com/amp/s/www.legacy.com/ob...
18,Dr. John D. Marshall Jr.,JOHN,MARSHALL,D.,JR.,NaN,DR.,74,08-15-2020,GA,https://www.ajc.com/news/first-georgia-doctor-...
19,Dr. Abelardo Vargas,ABELARDO,VARGAS,NaN,NaN,NaN,DR.,94,08-15-2020,None,https://medicinaysaludpublica.com/reportan-fal...
24,Dr. Gaye Griffin-Snyder,GAYE,GRIFFIN-SNYDER,NaN,NaN,NaN,DR.,71,08-09-2020,MO,https://www.stlouiscovidmemorial.com/dr-gaye-g...
25,Dr. Jorge Vallejo,JORGE,VALLEJO,NaN,NaN,NaN,DR.,None,08-08-2020,FL,https://m.facebook.com/groups/317464005892297?...
27,Dr. Thomas Rose,THOMAS,ROSE,NaN,NaN,NaN,DR.,60,08-08-2020,AL,https://www.google.com/amp/s/www.dothanfirst.c...


In [41]:
ppd[(ppd.LAST_NAME=='BOSSE')]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
138528,1103130464,A,1,2.0,RAPHAEL C BOSSE MD,BOSSE,RAPHAEL,CHARLES,NaN,NaN,...,NaN,NaN,NaN,NaN,BOSSE,NaN,RAPHAEL,NaN,NaN,NaN
230529,1643870149,A,1,1.0,BRUCE E BOSSE MD,BOSSE,BRUCE,ERIC,NaN,NaN,...,GA,30076,4919.0,C054,NaN,NaN,NaN,NaN,NaN,NaN
232117,1643991311,A,1,1.0,MATTHEW J BOSSE MD,BOSSE,MATTHEW,JAMES,NaN,THIRD FLOOR,...,CA,90028,6213.0,C061,NaN,NaN,NaN,NaN,NaN,NaN
265242,1875031704,A,1,2.0,ROBERT W BOSSE DO,BOSSE,ROBERT,WESLEY,NaN,NaN,...,MN,56001,4752.0,C026,NaN,NaN,NaN,NaN,NaN,NaN
287506,2002830186,A,1,1.0,GEORGE M BOSSE MD,BOSSE,GEORGE,MICHAEL,NaN,U OF LOUISVILLE DEPT EMER,...,KY,40202,1675.0,C202,BOSSE,MICHAEL,GEO,NaN,NaN,NaN
326562,2301780195,A,1,2.0,MICHAEL J BOSSE MD,BOSSE,MICHAEL,JOS,NaN,NaN,...,NC,28204,2966.0,C001,NaN,NaN,NaN,NaN,NaN,NaN
488836,3101160628,A,1,1.0,BRADLEY B BOSSE MD,BOSSE,BRADLEY,BERNARD,NaN,NaN,...,CA,92868,5900.0,C052,NaN,NaN,NaN,NaN,NaN,NaN
506803,3306870105,A,1,2.0,CHRISTOPHER G BOSSE MD,BOSSE,CHRISTOPHER,GERARD,NaN,NaN,...,PA,19001,3840.0,C001,NaN,NaN,NaN,NaN,NaN,NaN
506881,3306871098,A,1,2.0,MYHANH BOSSE MD,BOSSE,MYHANH,NaN,NaN,NaN,...,PA,18901,2503.0,C004,TANG,NaN,MYHANH,BOSSE,NaN,MYHANH
525924,3503151749,A,1,1.0,SAMANTHA BOSSE MD,BOSSE,SAMANTHA,NaN,NaN,DEPT OF FAMILY MED LOWER LEVEL,...,MD,21201,1771.0,C026,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
ppd.head(20)

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,102000013,A,1,2.0,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102000021,A,1,3.0,MERI K MARTIN MD,MARTIN,MERI,KING,NaN,NaN,...,NC,27617,7408.0,R077,NaN,NaN,NaN,NaN,NaN,NaN
2,102000030,A,1,1.0,MARK T JACKSON MD,JACKSON,MARK,THOMAS,NaN,NaN,...,NC,28805,1262.0,C082,NaN,NaN,NaN,NaN,NaN,NaN
3,102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,AZ,85375,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN
4,102000056,A,1,2.0,RAJIV B GALA MD,GALA,RAJIV,BABULAL,NaN,NaN,...,LA,70115,6949.0,C003,NaN,NaN,NaN,NaN,NaN,NaN
5,102000064,A,1,2.0,CAROL L ADAMS MD,ADAMS,CAROL,LYNN,NaN,NaN,...,AL,35504,2516.0,C009,DANIELS,LYNN,CAROL,NaN,NaN,NaN
6,102000072,A,1,2.0,JAMES H BANKSTON JR MD,BANKSTON,JAMES,HAROLD,2.0,NaN,...,AL,35406,1800.0,C057,NaN,NaN,NaN,NaN,NaN,NaN
7,102000081,A,1,2.0,ANDREA B MCMURPHY MD,MCMURPHY,ANDREA,BARBER,NaN,NaN,...,AL,36606,2457.0,C022,BARBER,ELIZABETH,ANDREA,NaN,NaN,NaN
8,102000099,A,1,2.0,MARY L GREGORY MD,GREGORY,MARY,LUCIA,NaN,NaN,...,TN,37232,34.0,C000,PARTIN,LUCIA,MARY,NaN,NaN,NaN
9,102000111,A,1,2.0,BARON C MAZE MD,MAZE,BARON,CHRISTOPHER,NaN,NaN,...,AL,35976,7140.0,R007,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ppd[(ppd.FIRST_NAME=='ANGEL')&(ppd.LAST_NAME=='GARCIA')]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
862354,4813081119,A,1,2.0,ANGEL J GARCIA MD,GARCIA,ANGEL,JOEL,NaN,NaN,...,TX,79924,4402.0,C024,NaN,NaN,NaN,NaN,NaN,NaN
1170246,64914791896,A,1,2.0,ANGEL M GARCIA MD,GARCIA,ANGEL,MARTY,NaN,# 4254,...,FL,33433,NaN,C048,GARCIA,MARY,ANGELA,NaN,NaN,NaN
1246185,84701630016,A,1,2.0,ANGEL R GARCIA MD,GARCIA,ANGEL,R,NaN,NaN,...,NaN,NaN,NaN,NaN,GARCIA-MARIN,R,ANGEL,NaN,NaN,NaN
